# Load Libraries

In [17]:
import warnings
warnings.filterwarnings('ignore')

import os
import time
import joblib
import json
import csv

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from collections import Counter
from pprint import pprint
%matplotlib inline

# Preprocessing
from sklearn.preprocessing import (
    StandardScaler,
    scale
    )
from sklearn.model_selection import ( 
    KFold,
    StratifiedKFold
    ) 
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    roc_curve,
    roc_auc_score, 
    precision_recall_curve,
    auc,
    precision_score, 
    recall_score, 
    f1_score
    )

from sklearn import metrics

# Keras
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers

import tensorflow as tf
from tensorflow.keras.callbacks import Callback

from functions_model import get_CNN_model, history_loss_acc, evaluate_matrix, ROC_PR_curve

,dataset,filepath,filename,age,gender,label,status,duration,duration_segment,sample_frequency,...,2806,2807,2808,2809,2810,2811,2812,2813,2814,2815
0,esc50,Dataset/ESC-50-master/ESC-50-master/audio/2-73...,2-73260-A-10,0.0,0,0,0,5.000000,1.0,22050,...,5.978531e-02,2.615242e-02,1.454682e-02,1.047784e-01,1.864351e-01,1.453094e-01,1.713713e-02,3.186968e-02,1.535154e-01,1.656509e-01
1,coughvid,Dataset/coughvid/6f0a2344-8d8b-4db5-b4c3-cf569...,6f0a2344-8d8b-4db5-b4c3-cf569ca4f7b9.wav,49.0,male,1,COVID-19,8.220000,1.0,22050,...,2.798310e-05,8.901093e-06,2.895766e-06,1.318840e-07,4.596973e-08,1.749610e-08,1.339242e-08,1.341929e-06,2.508061e-06,1.409820e-03
2,coughvid,Dataset/coughvid/b97e6429-d7e6-45fb-badf-49e52...,b97e6429-d7e6-45fb-badf-49e52d85d030.wav,0.0,0,0,0,9.840000,1.0,22050,...,3.274282e-08,1.811613e-08,7.445976e-08,2.263611e-08,1.163220e-08,1.622580e-08,1.739210e-08,3.316624e-12,1.128431e-37,7.547336e-10
3,fsdkaggle,Dataset/FSDKaggle2018/FSDKaggle2018.audio_trai...,c66094de,0.0,0,0,0,16.860000,1.0,22050,...,8.689960e-06,1.853306e-05,2.552348e-05,3.542432e-05,1.198441e-04,6.339062e-05,1.266576e-04,9.642727e-05,3.226967e-05,7.529062e-05
4,fsdkaggle,Dataset/FSDKaggle2018/FSDKaggle2018.audio_trai...,44f9acf9,0.0,0,0,0,14.780000,1.0,22050,...,1.343845e-04,1.364622e-03,7.193934e-04,2.822869e-05,6.511133e-03,1.850729e-04,8.441024e-04,3.963575e-04,5.704202e-05,4.064139e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,coswara,Dataset/Coswara-Data/Extracted_data/20200803/5...,5Gn8GDorBDaV9s4JhzMpppc3I3j1_cough-heavy,45.0,male,1,no_resp_illness_exposed,8.106667,1.0,22050,...,2.095462e-36,2.095462e-36,2.095462e-36,2.095462e-36,2.095462e-36,2.095462e-36,2.095462e-36,2.095462e-36,2.095462e-36,1.237004e-08
1996,fsdkaggle,Dataset/FSDKaggle2018/FSDKaggle2018.audio_trai...,a70a5165,0.0,0,1,0,13.000000,1.0,22050,...,6.599355e-08,7.011929e-08,3.246572e-07,1.019144e-07,7.102659e-08,5.674248e-08,6.087129e-08,8.604128e-08,9.005176e-08,1.213055e-07
1997,coswara,Dataset/Coswara-Data/Extracted_data/20210630/i...,iOx0zPyPiMZjCWXuf3RGu8HKq8k1_cough-shallow,39.0,male,1,positive_mild,3.584036,1.0,22050,...,7.848601e-05,2.000064e-04,1.181125e-36,1.181125e-36,1.181125e-36,1.181125e-36,1.181125e-36,1.181125e-36,1.181125e-36,5.276077e-09
1998,coswara,Dataset/Coswara-Data/Extracted_data/20200525/E...,ETcnGsA5gNUC1t3cXJY6KXLvF7W2_cough-heavy,49.0,male,1,healthy,7.616145,1.0,22050,...,1.044910e-03,5.864761e-04,6.886056e-04,1.302159e-03,6.904940e-04,4.668747e-04,4.311872e-04,3.169482e-04,9.676027e-04,5.043625e-04


# Load Data

In [20]:
list_datasets = [
    # ['fsdkaggle'],    # 2% cough Counter({0: 1570, 1: 30})
    # # # ['virufy'],       # 100% cough Counter({1: 121})
    # ['esc50'],        # 2% cough Counter({0: 1960, 1: 40})
    # ['coughvid'],     # 30% cough Counter({1: 19777, 0: 10267})
    # ['coswara'],      # 25% cough Counter({0: 18914, 1: 5408})
    ['coswara', 'coughvid', 'esc50', 'fsdkaggle', 'virufy'], 
]

overlap = 0

for window_length in [1, 5, 10]:

    df_results = []
    for datasets in list_datasets:
        datasets.sort()
        print('')
        print('#'*60)
        print(', '.join(datasets))
        print('Window length:', window_length)
        print('#'*60)
        
        dataset_str = '_'.join(datasets)

        path_model_save = f'Results/Model_CNN/{dataset_str}/CNN_{window_length}s/'
        
        if not os.path.exists(path_model_save):
            os.makedirs(path_model_save)
        
        ############################################################
        # Load data
        ############################################################
        df_all_combined = pd.DataFrame()
        for dataset in datasets:    
            df = pd.read_csv(f'Results/Features_CNN/data_{dataset}_features_{window_length}s_{overlap}.csv')
            df_all_combined = pd.concat([df_all_combined, df], axis=0)
        df_all_combined = df_all_combined.reset_index(drop=True)
        
        # df_all_combined = df_all_combined.fillna(df.mean())
        df_all_combined = df_all_combined.fillna(0)
        
        ############################################################
        # Get label distribution
        ############################################################
        df_all_combined = df_all_combined[df_all_combined['mean_amplitude'] > 0.005].reset_index(drop=True)
        df_all_combined = df_all_combined.sample(frac=1).groupby('label').head(1000).reset_index(drop=True)
        list_labels = df_all_combined['label'].tolist()
        count_labels = dict(Counter(list_labels))
        pprint(count_labels)
    
        ############################################################
        # Get features and labels
        ############################################################
        y = df_all_combined['label'].tolist()
        X = df_all_combined.drop(columns=['label'])
        
        ############################################################
        # Performance Store
        ############################################################
        list_cm = 0
        list_roc_auc, list_pr_auc = [], []
        list_pre, list_rec, list_f1 = [], [], []
        list_acc, list_spe, list_sen = [], [], []
        
        ############################################################
        # K-fold Cross Validation model evaluation
        ############################################################
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        fold_idx = 1
        for train_ids, test_ids in kfold.split(X, y):
            
            ############################################################
            # Get dataset split
            ############################################################
            df_train = X.loc[train_ids]
            y_train = np.array(y)[train_ids]
    
            df_test = X.loc[test_ids]
            y_test = np.array(y)[test_ids]
    
            ############################################################
            # Drop useless columns
            ############################################################
            def drop_columns(df):
                columns = [
                    'dataset', 'filename', 'filepath', 'age', 'gender', 'status',
                    'duration', 'duration_segment', 'sample_frequency', 'mean_amplitude',
                    'segment_shape',
                    ]
                for col in columns:
                    if col in df.columns:
                        df = df.drop([col], axis=1)
                
                df = np.array(df)
                return df
    
            X_train = drop_columns(df_train)
            X_test = drop_columns(df_test)


            ############################################################
            # Scaling
            ############################################################
            scaler = StandardScaler()
    
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
    
            # Save the scaler to a file
            scaler_filename = f"{path_model_save}scaler_pipeline_CNN_{fold_idx}.joblib"
            joblib.dump(scaler, scaler_filename)
    
            ############################################################
            # Oversampling (if required)
            ############################################################
            if datasets in [['fsdkaggle'], ['esc50']]:
                try:
                    oversample = SMOTE(sampling_strategy=0.5, k_neighbors=5)
                    X_train, y_train = oversample.fit_resample(X_train, y_train)
                except:
                    oversample = SMOTE(sampling_strategy=0.5, k_neighbors=3)
                    X_train, y_train = oversample.fit_resample(X_train, y_train)
    
            ############################################################
            # Reshaping
            ############################################################
            dimension_dictionary = {
                1: 22,
                5: 27,
                10: 27,
            }

            dim_first = 128
            input_shape = (dim_first, dimension_dictionary[window_length], 1)
            X_train = X_train.reshape((len(X_train), dim_first,  dimension_dictionary[window_length]))
            X_test = X_test.reshape((len(X_test), dim_first,  dimension_dictionary[window_length]))
    
            # Add one more axis for CNN
            X_train = X_train[..., np.newaxis]
            X_test = X_test[..., np.newaxis]
            
            ############################################################
            # Create Model
            ############################################################
            model = get_CNN_model(input_shape)
            # model.summary()
    
            batch_size = 16
            early_stopping_patience = 10

            # Add early stopping
            my_callbacks = [
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=path_model_save + 'Checkpoints/model_{epoch:02d}_' + f'CNN_{fold_idx}.keras', 
                    save_freq='epoch', 
                    save_best_only=True
                    ),
                tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss", 
                    patience=early_stopping_patience, 
                    restore_best_weights=True
                    )
            ]
    
            # Fit Model
            history = model.fit(
                X_train, y_train,
                epochs=100,
                batch_size=batch_size,
                callbacks=my_callbacks,
                validation_split=0.15,
                verbose=0,
                )
    
            history_loss_acc(history)
    
            test_loss, test_acc = model.evaluate(X_test, y_test)
            print('Test Accuracy: ', round(test_acc, 3))
    
            predictions = model.predict(X_test)
            y_predict = []
            for i in range(len(predictions)):
                predict = np.argmax(predictions[i])
                y_predict.append(predict)
                        
            ############################################################
            # Append predictions to df_test and save
            ############################################################
            df_test['true'] = y_test
            df_test['pred'] = y_predict
            
            path_results_save = f'Results/Results CNN/{dataset_str}/CNN_{window_length}s/'
            if not os.path.exists(path_results_save):
                os.makedirs(path_results_save)
            
            df_test.to_csv(f'{path_results_save}Fold_{fold_idx}.csv', index=False)
            
            ############################################################
            # Get evaluation metrics
            ############################################################
            acc = accuracy_score(y_test, y_predict)
            cm = evaluate_matrix(y_test, y_predict)
            roc_auc, pr_auc = ROC_PR_curve(y_test, predictions)
            pre = precision_score(y_test, y_predict)
            rec = recall_score(y_test, y_predict)
            f1 = f1_score(y_test, y_predict)
            tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
            spe = tn / (tn + fp)
            sen = rec
            
            ############################################################
            # Append results
            ############################################################
            list_acc.append(acc)
            list_cm = list_cm + cm
            list_roc_auc.append(roc_auc)
            list_pr_auc.append(pr_auc)
            list_pre.append(pre)
            list_rec.append(rec)
            list_f1.append(f1)
            list_spe.append(spe)
            list_sen.append(sen)

            print(f"Fold {fold_idx} - F1: {round(f1, 3)}")
            
            ############################################################
            # Save model
            ############################################################
            # Serialize model to JSON
            model_json = model.to_json()
            with open(f"{path_model_save}model_{fold_idx}.json", "w") as json_file:
                json_file.write(model_json)
            
            model.save_weights(f"{path_model_save}model_{fold_idx}.weights.h5")
            model.save(f"{path_model_save}model_{fold_idx}.h5")
            # loaded_model = get_model()
            # loaded_model.load_weights('Results/Model/kfold.h5')
    
            ############################################################
            # Save results
            ############################################################
            results = [
                ', '.join(datasets), count_labels, 
                window_length, overlap,
                'CNN', fold_idx,
                acc, sen, spe, pre, rec, f1, roc_auc, pr_auc, cm]
            df_results.append(results)
    
            # To the next fold
            fold_idx = fold_idx + 1
            
        results = [
            ', '.join(datasets),  count_labels,
            window_length, overlap,
            'CNN', 'Avg',
            np.mean(list_acc),
            np.mean(list_sen),
            np.mean(list_spe),
            np.mean(list_pre),
            np.mean(list_rec),
            np.mean(list_f1),
            np.mean(list_roc_auc),
            np.mean(list_pr_auc),
            list_cm]
        df_results.append(results)
        
        print(list_cm)
        print(f'ROC AUC: {np.mean(list_roc_auc)}')
        print(f'PR AUC: {np.mean(list_pr_auc)}')
        print(f'F1: {np.mean(list_f1)}')
    
    columns = ['dataset', 'label_count', 'window_length', 'overlap',
               'model', 'fold', 
               'acc', 'sen', 'spe', 'pre', 'rec', 'f1', 'auc', 'auprc', 'cm']    
    df_results = pd.DataFrame(df_results, columns = columns)
    df_results.to_csv(f'Results/Model_CNN/results_prediction_CNN_{window_length}s_{overlap}.csv', index=False)

print('#'*60)
print('DONE')
print('#'*60)


############################################################
coswara, coughvid, esc50, fsdkaggle, virufy
Window length: 1
############################################################
{0: 1000, 1: 1000}


2025-09-18 02:02:32.938352: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7774 - loss: 0.8186
Test Accuracy:  0.803
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Fold 1 - F1: 0.815


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7898 - loss: 0.8215
Test Accuracy:  0.812
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step


Fold 2 - F1: 0.818
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.8358 - loss: 0.6482
Test Accuracy:  0.837
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7794 - loss: 0.6562
Test Accuracy:  0.798
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Fold 2 - F1: 0.801


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.8233 - loss: 0.8206
Test Accuracy:  0.822
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Fold 3 - F1: 0.816


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.7146 - loss: 2.4042
Test Accuracy:  0.745
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
Fold 4 - F1: 0.771


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7834 - loss: 1.7686
Test Accuracy:  0.79
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step


Fold 5 - F1: 0.805
[[709 291]
 [141 859]]
ROC AUC: 0.8339000000000001
PR AUC: 0.7842655523290187
F1: 0.7994996161800881
############################################################
DONE
############################################################


# Combine all files

In [21]:
# Create an empty DataFrame to hold the combined data
combined_df = pd.DataFrame()

# Loop through each file and concatenate
for window_length in [1, 5, 10]:
    df = pd.read_csv(f'Results/Model_CNN/results_prediction_CNN_{window_length}s_{overlap}.csv')
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Display or save the result
print(combined_df)
combined_df.to_csv(f'Results/Model_CNN/results_prediction_CNN_All.csv', index=False)

                                        dataset         label_count  \
0   coswara, coughvid, esc50, fsdkaggle, virufy  {1: 1000, 0: 1000}   
1   coswara, coughvid, esc50, fsdkaggle, virufy  {1: 1000, 0: 1000}   
2   coswara, coughvid, esc50, fsdkaggle, virufy  {1: 1000, 0: 1000}   
3   coswara, coughvid, esc50, fsdkaggle, virufy  {1: 1000, 0: 1000}   
4   coswara, coughvid, esc50, fsdkaggle, virufy  {1: 1000, 0: 1000}   
5   coswara, coughvid, esc50, fsdkaggle, virufy  {1: 1000, 0: 1000}   
6   coswara, coughvid, esc50, fsdkaggle, virufy  {0: 1000, 1: 1000}   
7   coswara, coughvid, esc50, fsdkaggle, virufy  {0: 1000, 1: 1000}   
8   coswara, coughvid, esc50, fsdkaggle, virufy  {0: 1000, 1: 1000}   
9   coswara, coughvid, esc50, fsdkaggle, virufy  {0: 1000, 1: 1000}   
10  coswara, coughvid, esc50, fsdkaggle, virufy  {0: 1000, 1: 1000}   
11  coswara, coughvid, esc50, fsdkaggle, virufy  {0: 1000, 1: 1000}   
12  coswara, coughvid, esc50, fsdkaggle, virufy  {1: 1000, 0: 1000}   
13  co